In [1]:
import os
import json
from tqdm import tqdm

def convert_json_to_txt(json_folder, output_folder):
    # JSON 파일 목록 가져오기
    json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

    for json_file in tqdm(json_files, desc='Converting JSON to YOLO txt'):
        with open(os.path.join(json_folder, json_file), 'r') as f:
            data = json.load(f)
        
        # 이미지 크기 가져오기
        img_width = data['description']['width']
        img_height = data['description']['height']

        # YOLO 형식으로 변환
        yolo_annotations = []
        for i in range(len(data['annotations']['points'])):
            disease_id = data['annotations']['disease']
            if disease_id == 0:
                continue
            elif disease_id == 16:
                class_id = 0
            else:
                class_id = 1
            x = data['annotations']['points'][i]['xtl']
            y = data['annotations']['points'][i]['ytl']
            width = data['annotations']['points'][i]['xbr'] - x
            height = data['annotations']['points'][i]['ybr'] - y

            # 좌표 정규화
            x_center = (x + width/2) / img_width
            y_center = (y + height/2) / img_height
            norm_width = width / img_width
            norm_height = height / img_height

            yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}")
        
        # YOLO 형식으로 저장
        img_name = json_file.split('.')[0]
        output_file = os.path.join(output_folder, img_name+'.txt')
        with open(output_file, 'w') as f:
            f.write('\n'.join(yolo_annotations))

In [2]:
json_folder = "json_files_path"
output_folder = "output_path"
convert_json_to_txt(json_folder, output_folder)

Converting JSON to YOLO txt:   0%|          | 0/1148 [00:00<?, ?it/s]

Converting JSON to YOLO txt: 100%|██████████| 1148/1148 [00:11<00:00, 103.63it/s]
